In [3]:
import pandas as pd
import numpy as np
import gc
path='/home/student303/oyx/almama/'
train = pd.read_hdf(path+ 'somefeature/base_feature_train_0.h5')
test = pd.read_hdf(path+ 'somefeature/base_feature_test_0.h5')
print('Read Data Finish')

Read Data Finish


In [4]:
combine = [train, test]
co_df = pd.concat(combine, axis=0, ignore_index=True)

In [89]:
for j in list(train.columns):
    print(co_df[~co_df[j].duplicated()].shape, end='*****')
    print(j)

(496455, 32)*****instance_id
(10236, 32)*****item_id
(14, 32)*****item_category_list
(11181, 32)*****item_property_list
(2075, 32)*****item_brand_id
(128, 32)*****item_city_id
(14, 32)*****item_price_level
(18, 32)*****item_sales_level
(18, 32)*****item_collected_level
(22, 32)*****item_pv_level
(207641, 32)*****user_id
(4, 32)*****user_gender_id
(9, 32)*****user_age_level
(5, 32)*****user_occupation_id
(12, 32)*****user_star_level
(496482, 32)*****context_id
(297416, 32)*****context_timestamp
(20, 32)*****context_page_id
(82378, 32)*****predict_category_property
(4003, 32)*****shop_id
(25, 32)*****shop_review_num_level
(13021, 32)*****shop_review_positive_rate
(22, 32)*****shop_star_level
(18102, 32)*****shop_score_service
(18151, 32)*****shop_score_delivery
(18204, 32)*****shop_score_description
(3, 32)*****is_trade
(14, 32)*****item_category_hash
(8, 32)*****day
(24, 32)*****hour
(60, 32)*****min
(60, 32)*****second


In [5]:
co_df_columns = list(co_df.columns)
for i in ['instance_id',
          'context_id',
          'context_timestamp',
          'predict_category_property',
          'is_trade',
          'day',
          
          'shop_score_service',
          'shop_score_delivery',
          'shop_score_description',
          'shop_review_positive_rate'
         ]:
    co_df_columns.remove(i)

In [6]:
co_df_columns

['context_page_id',
 'hour',
 'item_brand_id',
 'item_category_hash',
 'item_category_list',
 'item_city_id',
 'item_collected_level',
 'item_id',
 'item_price_level',
 'item_property_list',
 'item_pv_level',
 'item_sales_level',
 'min',
 'second',
 'shop_id',
 'shop_review_num_level',
 'shop_star_level',
 'user_age_level',
 'user_gender_id',
 'user_id',
 'user_occupation_id',
 'user_star_level']

In [7]:
name = locals()
for starttime in range(18,23):
    windowsize = 3
    windowday = []
    for i in range(windowsize):
        windowday.append(starttime + i)
    tmpdata = pd.DataFrame()  
    for i in windowday:
        tmpdata = pd.concat([tmpdata,co_df[co_df.day == i]], axis=0, ignore_index=True)
    name['merge_2' + '_' + str(starttime + windowsize)] = co_df[co_df.day == (starttime + windowsize)]
    
    for m in range(len(co_df_columns)):
        for n in range(m+1,len(co_df_columns)):
            tmpmerge1 = tmpdata[[co_df_columns[m],co_df_columns[n],'instance_id']].groupby([co_df_columns[m],co_df_columns[n]]).count().reset_index()
            tmpmerge1.rename(columns={'instance_id':'A'}, inplace = True)

            tmpmerge2 = tmpdata[tmpdata.is_trade == 1][[co_df_columns[m],co_df_columns[n],'instance_id']].groupby([co_df_columns[m],co_df_columns[n]]).count().reset_index()
            tmpmerge2.rename(columns={'instance_id':'B'}, inplace = True)

            tmpmerge1 = pd.merge(tmpmerge1, tmpmerge2, on=[co_df_columns[m],co_df_columns[n]], how='left')
            tmpmerge1 = tmpmerge1.fillna(0)
            
            tmpmerge1[co_df_columns[m]+ '->'+co_df_columns[n] + '_cvr'] = (tmpmerge1['B']+1) / (tmpmerge1['A']+50)
            tmpmerge1.drop(['A','B'], axis=1, inplace=True)
            name['merge_2' +  '_' +str(starttime + windowsize)] = pd.merge(name['merge_2' +  '_' +str(starttime + windowsize)] , tmpmerge1, on=[co_df_columns[m],co_df_columns[n]], how='left')
            print('we done '+ co_df_columns[m] +'->'+ co_df_columns[n] + '_cvr')
            del tmpmerge1
            del tmpmerge2 
            gc.collect()

we done context_page_id->hour_cvr
we done context_page_id->item_brand_id_cvr
we done context_page_id->item_category_hash_cvr
we done context_page_id->item_category_list_cvr
we done context_page_id->item_city_id_cvr
we done context_page_id->item_collected_level_cvr
we done context_page_id->item_id_cvr
we done context_page_id->item_price_level_cvr
we done context_page_id->item_property_list_cvr
we done context_page_id->item_pv_level_cvr
we done context_page_id->item_sales_level_cvr
we done context_page_id->min_cvr
we done context_page_id->second_cvr
we done context_page_id->shop_id_cvr
we done context_page_id->shop_review_num_level_cvr
we done context_page_id->shop_star_level_cvr
we done context_page_id->user_age_level_cvr
we done context_page_id->user_gender_id_cvr
we done context_page_id->user_id_cvr
we done context_page_id->user_occupation_id_cvr
we done context_page_id->user_star_level_cvr
we done hour->item_brand_id_cvr
we done hour->item_category_hash_cvr
we done hour->item_categor

we done item_property_list->user_occupation_id_cvr
we done item_property_list->user_star_level_cvr
we done item_pv_level->item_sales_level_cvr
we done item_pv_level->min_cvr
we done item_pv_level->second_cvr
we done item_pv_level->shop_id_cvr
we done item_pv_level->shop_review_num_level_cvr
we done item_pv_level->shop_star_level_cvr
we done item_pv_level->user_age_level_cvr
we done item_pv_level->user_gender_id_cvr
we done item_pv_level->user_id_cvr
we done item_pv_level->user_occupation_id_cvr
we done item_pv_level->user_star_level_cvr
we done item_sales_level->min_cvr
we done item_sales_level->second_cvr
we done item_sales_level->shop_id_cvr
we done item_sales_level->shop_review_num_level_cvr
we done item_sales_level->shop_star_level_cvr
we done item_sales_level->user_age_level_cvr
we done item_sales_level->user_gender_id_cvr
we done item_sales_level->user_id_cvr
we done item_sales_level->user_occupation_id_cvr
we done item_sales_level->user_star_level_cvr
we done min->second_cvr
we 

we done item_id->min_cvr
we done item_id->second_cvr
we done item_id->shop_id_cvr
we done item_id->shop_review_num_level_cvr
we done item_id->shop_star_level_cvr
we done item_id->user_age_level_cvr
we done item_id->user_gender_id_cvr
we done item_id->user_id_cvr
we done item_id->user_occupation_id_cvr
we done item_id->user_star_level_cvr
we done item_price_level->item_property_list_cvr
we done item_price_level->item_pv_level_cvr
we done item_price_level->item_sales_level_cvr
we done item_price_level->min_cvr
we done item_price_level->second_cvr
we done item_price_level->shop_id_cvr
we done item_price_level->shop_review_num_level_cvr
we done item_price_level->shop_star_level_cvr
we done item_price_level->user_age_level_cvr
we done item_price_level->user_gender_id_cvr
we done item_price_level->user_id_cvr
we done item_price_level->user_occupation_id_cvr
we done item_price_level->user_star_level_cvr
we done item_property_list->item_pv_level_cvr
we done item_property_list->item_sales_level

we done item_city_id->item_sales_level_cvr
we done item_city_id->min_cvr
we done item_city_id->second_cvr
we done item_city_id->shop_id_cvr
we done item_city_id->shop_review_num_level_cvr
we done item_city_id->shop_star_level_cvr
we done item_city_id->user_age_level_cvr
we done item_city_id->user_gender_id_cvr
we done item_city_id->user_id_cvr
we done item_city_id->user_occupation_id_cvr
we done item_city_id->user_star_level_cvr
we done item_collected_level->item_id_cvr
we done item_collected_level->item_price_level_cvr
we done item_collected_level->item_property_list_cvr
we done item_collected_level->item_pv_level_cvr
we done item_collected_level->item_sales_level_cvr
we done item_collected_level->min_cvr
we done item_collected_level->second_cvr
we done item_collected_level->shop_id_cvr
we done item_collected_level->shop_review_num_level_cvr
we done item_collected_level->shop_star_level_cvr
we done item_collected_level->user_age_level_cvr
we done item_collected_level->user_gender_id_c

we done item_category_hash->shop_review_num_level_cvr
we done item_category_hash->shop_star_level_cvr
we done item_category_hash->user_age_level_cvr
we done item_category_hash->user_gender_id_cvr
we done item_category_hash->user_id_cvr
we done item_category_hash->user_occupation_id_cvr
we done item_category_hash->user_star_level_cvr
we done item_category_list->item_city_id_cvr
we done item_category_list->item_collected_level_cvr
we done item_category_list->item_id_cvr
we done item_category_list->item_price_level_cvr
we done item_category_list->item_property_list_cvr
we done item_category_list->item_pv_level_cvr
we done item_category_list->item_sales_level_cvr
we done item_category_list->min_cvr
we done item_category_list->second_cvr
we done item_category_list->shop_id_cvr
we done item_category_list->shop_review_num_level_cvr
we done item_category_list->shop_star_level_cvr
we done item_category_list->user_age_level_cvr
we done item_category_list->user_gender_id_cvr
we done item_category

In [8]:
tmpdf_2 =  pd.DataFrame() 
for i in range(21,26):
    tmpdf_2 = pd.concat([tmpdf_2,name['merge_2'+'_'+str(i)]], axis=0, ignore_index=True)

## save data to somefeature/two_column_cvr.h5

In [9]:
save_train = tmpdf_2[tmpdf_2.day != 25]
save_test = tmpdf_2[tmpdf_2.day == 25]

In [10]:
tmpdf_2.to_hdf(path + 'somefeature/two_column_cvr.h5',key='a',index=False)
save_train.to_hdf(path + 'somefeature/two_column_train_cvr.h5',key='a',index=False)
save_test.to_hdf(path + 'somefeature/two_column_test_cvr.h5',key='a',index=False)

## test program

## 

In [69]:
co_df[['hour','min','is_trade','day']][co_df.hour == 23][co_df['min'] == 33][co_df.day >= 18][co_df.day <= 20].shape

/home/student303/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(153, 4)

In [70]:
co_df[['hour','min','is_trade','day']][co_df.hour == 23][co_df['min'] == 33][co_df.day >= 18][co_df.day <= 20][co_df.is_trade == 1].shape

/home/student303/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(0, 4)

In [59]:
test1[test1['min'] == 37].shape

(445, 4)

In [60]:
test1[test1['min'] == 37][test1.is_trade == 1].shape

(7, 4)

In [68]:
6/318

0.018867924528301886

In [39]:
test1 = tmpdf_2[['hour->min_cvr','is_trade','day','min','hour','instance_id']]

In [40]:
test1[test1.day == 21]

,hour->min_cvr,is_trade,day,min,hour,instance_id
0,NaN,0.0,21,52,1,4293715837363757697
1,0.018868,0.0,21,37,21,70764105903083828
2,0.017483,0.0,21,14,22,3954908518456917085
3,0.015152,0.0,21,53,13,4521314384240415059
4,0.006173,0.0,21,51,9,6005676298575614673
5,0.020513,0.0,21,47,18,655162606126973882
6,NaN,1.0,21,33,23,6388321779120371962
7,NaN,0.0,21,51,2,2909190494355726720
8,0.026627,0.0,21,10,21,8631039197689382476
9,0.009132,0.0,21,9,23,1363985136028256226


In [167]:
tmpdf =  pd.DataFrame() 
for i in range(21,25):
    tmpdf = pd.concat([tmpdf,name['merge_2'+'_'+str(i)]], axis=0, ignore_index=True)

co_df_columns = ['hour','item_brand_id']
starttime = 19
windowsize = 3
tmpdata = pd.DataFrame() 
for i in [19,20,21]:
    tmpdata = pd.concat([tmpdata,co_df[co_df.day == i]], axis=0, ignore_index=True)
name['merge_2' + '_' + str(starttime + windowsize)] = co_df[co_df.day == (starttime + windowsize)]
for m in range(len(co_df_columns)):
    for n in range(m+1,len(co_df_columns)):
        tmpmerge1 = tmpdata[[co_df_columns[m],co_df_columns[n],'instance_id']].groupby([co_df_columns[m],co_df_columns[n]]).count().reset_index()
        tmpmerge1.rename(columns={'instance_id':'A'}, inplace = True)

        tmpmerge2 = tmpdata[tmpdata.is_trade == 1][[co_df_columns[m],co_df_columns[n],'instance_id']].groupby([co_df_columns[m],co_df_columns[n]]).count().reset_index()
        tmpmerge2.rename(columns={'instance_id':'B'}, inplace = True)
        
        tmpmerge1 = pd.merge(tmpmerge1, tmpmerge2, on=[co_df_columns[m],co_df_columns[n]], how='left')
        tmpmerge1[co_df_columns[m]+ '->'+co_df_columns[n] + '_cvr'] = tmpmerge1['B'] / tmpmerge1['A']
        #tmpmerge1.drop(['A','B'], axis=1, inplace=True)
       # name['merge' +  '_' +str(starttime + windowsize)] = pd.merge(name['merge' +  '_' +str(starttime + windowsize)] , tmpmerge1, on=[co_df_columns[m],co_df_columns[n]], how='left')